In [1]:
import numpy as np
import math
import copy

gamma = 0.6
N_e = 2

In [2]:
class State:
    ball_x = 0
    ball_y = 0
    velocity_x = 0
    velocity_y = 0
    paddle_y = 0
    terminate = False
    
    
    def print_state(self):
        print(np.around(self.ball_x, 2) , " " , np.around(self.ball_y, 2) , " " ,  
              np.around(self.velocity_x, 2) , " " ,  np.around(self.velocity_y, 2) , " " ,  np.around(self.paddle_y, 2))
              
 
    def __init__(self, bx, by, vx, vy, p):
        self.ball_x = bx
        self.ball_y = by
        self.velocity_x = vx
        self.velocity_y = vy
        self.paddle_y = p
        
         
    def up(self):
        self.paddle_y = max(self.paddle_y - 0.04, 0)
        
        self.update()
        #print("in function up", match(self))

    def down(self):
        self.paddle_y = min(self.paddle_y + 0.04, 0.8)
        self.update()

    def update(self):
        global num_bounce
#         global state_tracker
        
        self.ball_x += self.velocity_x
        self.ball_y += self.velocity_y
        
        if self.ball_x >= 1:        # hitting right bound
            
            
            if self.ball_y >= self.paddle_y and self.ball_y <= self.paddle_y + 0.2:    # hitting paddle
                self.ball_x = 2 - self.ball_x
                #print("bounce")
                num_bounce += 1
#                 if curr_state not in state_tracker:
#                     #print(curr_state)
#                     state_tracker.append(curr_state)
                                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
                    
                
                state_tracker = []
                
            else:    # missing paddle
                #print("terminate")
                self.ball_x = 2 - self.ball_x
#                 reward(state_tracker,-1)
#                 state_tracker = []
                self.terminate = True

            
        if self.ball_x <= 0:     # hitting left bound
            self.ball_x = -self.ball_x
            self.velocity_x = -self.velocity_x
        if self.ball_y <= 0:     # hitting upper bound
            self.ball_y = -self.ball_y
            self.velocity_y = -self.velocity_y
        if self.ball_y  >= 1:    # hitting right bound
            self.ball_y = 2 - self.ball_y
            self.velocity_y = -self.velocity_y

In [3]:
def match(s):
    b_x = math.floor(s.ball_x * 12)
    b_y = math.floor(s.ball_y * 12)
    if s.velocity_x > 0:
        v_x = 1
    else:
        v_x = 0    # velocity_x <= 0 map to 0

    v_y = 0
    if s.velocity_y > 0 and abs(s.velocity_y) > 0.015:
        v_y = 1
    elif s.velocity_y < 0 and abs(s.velocity_y) > 0.015:    #velocity_y < 0.015 map to 2
        v_y = 2

    #print("paddle_y", s.paddle_y)
    p = math.floor(s.paddle_y / (1 - 0.2) * 12)
    if p == 12:
        p = 11
    #print("p", p)

    return (b_x ,b_y, v_x, v_y, p)

In [4]:
def reward(state):
    next_ball_x = state.ball_x + state.velocity_x

    if next_ball_x >= 1:        # hitting right bound
        if state.ball_y >= state.paddle_y and state.ball_y <= state.paddle_y + 0.2:    # hitting paddle
            return 1
        else:     # missing paddle
            return -1
    return 0

In [5]:

def get_up(current_state):
    #print("up current", match(current_state))
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.up()
    #print("new state", match(new_state))
    return new_state

def get_down(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.down()
    return new_state

def get_no_move(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.update()
    return new_state

In [6]:
def max_Q(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    Q_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    #print(b_x ,b_y, v_x, v_y, p)
    Q_down = Q_table[b_x][b_y][v_x][v_y][p][1]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    Q_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    #print(Q_up, Q_down, Q_no_move)
    
    return max(Q_up, Q_down, Q_no_move)

In [7]:
def argmax_f(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][0] < N_e:
        f_up = 0
    else:
        f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][1] < N_e:
        f_down = 0
    else:
        f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][2] < N_e:
        f_no_move = 0
    else:
        f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [8]:
def alpha(state):
    (b_x ,b_y, v_x, v_y, p) = match(state)
    num_visited = N_sa[b_x][b_y][v_x][v_y][p][a] 
    return 60 / (60 + num_visited)

In [9]:
Q_table = np.zeros((12, 12, 2, 3, 12, 3)) # x, y, v_x, v_y, paddle, action
Q_additional_state = -1                   # x >= 1 state
N_sa = np.zeros((12, 12, 2, 3, 12, 3))    # x, y, v_x, v_y, paddle, action

s = State(0,0,0,0,0)
s_empty = True
a = 0
r = 0

In [10]:
'''action: 0-up, 1-down, 2-no_move'''
def Q_agent(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q
    if current_state.terminate:
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #Q_table[b_x][b_y][v_x][v_y][p][3] = reward(current_state)

    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #print(b_x ,b_y, v_x, v_y, p, a)
        N_sa[b_x][b_y][v_x][v_y][p][a] += 1
        #print(N_sa[b_x][b_y][v_x][v_y][p][a])
        
        Q_table[b_x][b_y][v_x][v_y][p][a] =  Q_table[b_x][b_y][v_x][v_y][p][a] + \
                            alpha(s)* (r + gamma * max_Q(current_state) - Q_table[b_x][b_y][v_x][v_y][p][a])
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f(current_state)
    
    r = reward(current_state)
    
    return a

In [ ]:

t_ = 1
num_bounce = 0
total_num_bounce = 0
for i in range(100):
    s = State(0.5, 0.5, 0.03, 0.01, 0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    while s.terminate is False:
        #s.print_state()
        next_action = Q_agent(current_state, t_)
        #print("current state before action", match(current_state))
        #print("next action", next_action)
        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))
# print(total_num_bounce / 10000)

round:  0
bounce: 9
state visited:  2
round:  1
bounce: 7
state visited:  5
round:  2
bounce: 0
state visited:  7
round:  3
bounce: 0
state visited:  8
round:  4
bounce: 17
state visited:  13
round:  5
bounce: 0
state visited:  14
round:  6
bounce: 0
state visited:  14
round:  7
bounce: 7
state visited:  16
round:  8
bounce: 8
state visited:  19
round:  9
bounce: 14
state visited:  23
round:  10
bounce: 4
state visited:  25
round:  11
bounce: 7
state visited:  28
round:  12
bounce: 4
state visited:  31
round:  13
bounce: 7
state visited:  34
round:  14
bounce: 5
state visited:  35
round:  15
bounce: 5
state visited:  36
round:  16
bounce: 4
state visited:  36
round:  17
bounce: 4
state visited:  36
round:  18
bounce: 4
state visited:  36
round:  19
bounce: 5
state visited:  37
round:  20
bounce: 5
state visited:  38
round:  21
bounce: 5
state visited:  39
round:  22
bounce: 5
state visited:  40
round:  23
bounce: 7
state visited:  41
round:  24
bounce: 4
state visited:  41
round:  25
b

bounce: 7
state visited:  289
round:  202
bounce: 7
state visited:  289
round:  203
bounce: 7
state visited:  290
round:  204
bounce: 16
state visited:  295
round:  205
bounce: 7
state visited:  298
round:  206
bounce: 7
state visited:  299
round:  207
bounce: 26
state visited:  308
round:  208
bounce: 7
state visited:  309
round:  209
bounce: 7
state visited:  312
round:  210
bounce: 12
state visited:  314
round:  211
bounce: 7
state visited:  314
round:  212
bounce: 7
state visited:  315
round:  213
bounce: 14
state visited:  319
round:  214
bounce: 7
state visited:  319
round:  215
bounce: 7
state visited:  320
round:  216
bounce: 7
state visited:  321
round:  217
bounce: 7
state visited:  322
round:  218
bounce: 7
state visited:  323
round:  219
bounce: 7
state visited:  324
round:  220
bounce: 12
state visited:  326
round:  221
bounce: 4
state visited:  326
round:  222
bounce: 7
state visited:  327
round:  223
bounce: 15
state visited:  332
round:  224
bounce: 33
state visited:  3

round:  404
bounce: 7
state visited:  785
round:  405
bounce: 10
state visited:  790
round:  406
bounce: 14
state visited:  793
round:  407
bounce: 14
state visited:  795
round:  408
bounce: 7
state visited:  799
round:  409
bounce: 7
state visited:  802
round:  410
bounce: 7
state visited:  805
round:  411
bounce: 7
state visited:  810
round:  412
bounce: 18
state visited:  817
round:  413
bounce: 7
state visited:  822
round:  414
bounce: 7
state visited:  824
round:  415
bounce: 12
state visited:  828
round:  416
bounce: 5
state visited:  829
round:  417
bounce: 5
state visited:  830
round:  418
bounce: 5
state visited:  831
round:  419
bounce: 12
state visited:  837
round:  420
bounce: 7
state visited:  838
round:  421
bounce: 5
state visited:  839
round:  422
bounce: 14
state visited:  848
round:  423
bounce: 12
state visited:  852
round:  424
bounce: 14
state visited:  858
round:  425
bounce: 7
state visited:  859
round:  426
bounce: 5
state visited:  861
round:  427
bounce: 7
sta

bounce: 12
state visited:  1453
round:  599
bounce: 7
state visited:  1455
round:  600
bounce: 12
state visited:  1460
round:  601
bounce: 7
state visited:  1461
round:  602
bounce: 5
state visited:  1463
round:  603
bounce: 14
state visited:  1471
round:  604
bounce: 17
state visited:  1478
round:  605
bounce: 12
state visited:  1483
round:  606
bounce: 8
state visited:  1489
round:  607
bounce: 28
state visited:  1497
round:  608
bounce: 7
state visited:  1505
round:  609
bounce: 5
state visited:  1508
round:  610
bounce: 21
state visited:  1515
round:  611
bounce: 7
state visited:  1516
round:  612
bounce: 5
state visited:  1520
round:  613
bounce: 5
state visited:  1526
round:  614
bounce: 5
state visited:  1529
round:  615
bounce: 5
state visited:  1530
round:  616
bounce: 12
state visited:  1539
round:  617
bounce: 5
state visited:  1540
round:  618
bounce: 24
state visited:  1565
round:  619
bounce: 7
state visited:  1568
round:  620
bounce: 7
state visited:  1569
round:  621
bo

bounce: 29
state visited:  2712
round:  795
bounce: 7
state visited:  2715
round:  796
bounce: 11
state visited:  2724
round:  797
bounce: 7
state visited:  2725
round:  798
bounce: 8
state visited:  2731
round:  799
bounce: 14
state visited:  2739
round:  800
bounce: 7
state visited:  2740
round:  801
bounce: 5
state visited:  2741
round:  802
bounce: 8
state visited:  2746
round:  803
bounce: 8
state visited:  2750
round:  804
bounce: 19
state visited:  2762
round:  805
bounce: 17
state visited:  2767
round:  806
bounce: 14
state visited:  2770
round:  807
bounce: 7
state visited:  2773
round:  808
bounce: 32
state visited:  2786
round:  809
bounce: 23
state visited:  2797
round:  810
bounce: 7
state visited:  2799
round:  811
bounce: 16
state visited:  2808
round:  812
bounce: 14
state visited:  2819
round:  813
bounce: 5
state visited:  2820
round:  814
bounce: 5
state visited:  2821
round:  815
bounce: 5
state visited:  2822
round:  816
bounce: 14
state visited:  2825
round:  817


...

bounce: 66
state visited:  10101
round:  2124
bounce: 19
state visited:  10101
round:  2125
bounce: 45
state visited:  10108
round:  2126
bounce: 7
state visited:  10108
round:  2127
bounce: 5
state visited:  10108
round:  2128
bounce: 43
state visited:  10123
round:  2129
bounce: 25
state visited:  10124
round:  2130
bounce: 12
state visited:  10128
round:  2131
bounce: 19
state visited:  10134
round:  2132
bounce: 10
state visited:  10136
round:  2133
bounce: 26
state visited:  10137
round:  2134
bounce: 50
state visited:  10147
round:  2135
bounce: 12
state visited:  10156
round:  2136
bounce: 12
state visited:  10156
round:  2137
bounce: 12
state visited:  10162
round:  2138
bounce: 14
state visited:  10162
round:  2139
bounce: 12
state visited:  10167
round:  2140
bounce: 12
state visited:  10170
round:  2141
bounce: 7
state visited:  10173
round:  2142
bounce: 12
state visited:  10173
round:  2143
bounce: 39
state visited:  10183
round:  2144
bounce: 10
state visited:  10192
roun

bounce: 12
state visited:  10778
round:  2306
bounce: 7
state visited:  10778
round:  2307
bounce: 15
state visited:  10780
round:  2308
bounce: 43
state visited:  10805
round:  2309
bounce: 32
state visited:  10808
round:  2310
bounce: 5
state visited:  10808
round:  2311
bounce: 12
state visited:  10813
round:  2312
bounce: 17
state visited:  10818
round:  2313
bounce: 55
state visited:  10820
round:  2314
bounce: 5
state visited:  10820
round:  2315
bounce: 15
state visited:  10822
round:  2316
bounce: 26
state visited:  10822
round:  2317
bounce: 5
state visited:  10823
round:  2318
bounce: 11
state visited:  10823
round:  2319
bounce: 33
state visited:  10826
round:  2320
bounce: 35
state visited:  10831
round:  2321
bounce: 23
state visited:  10839
round:  2322
bounce: 22
state visited:  10840
round:  2323
bounce: 7
state visited:  10842
round:  2324
bounce: 12
state visited:  10842
round:  2325
bounce: 5
state visited:  10842
round:  2326
bounce: 19
state visited:  10844
round: 

bounce: 48
state visited:  11312
round:  2489
bounce: 52
state visited:  11315
round:  2490
bounce: 33
state visited:  11320
round:  2491
bounce: 11
state visited:  11320
round:  2492
bounce: 19
state visited:  11322
round:  2493
bounce: 8
state visited:  11322
round:  2494
bounce: 7
state visited:  11322
round:  2495
bounce: 14
state visited:  11322
round:  2496
bounce: 13
state visited:  11330
round:  2497
bounce: 10
state visited:  11331
round:  2498
bounce: 36
state visited:  11332
round:  2499
bounce: 18
state visited:  11332
round:  2500
bounce: 29
state visited:  11334
round:  2501
bounce: 20
state visited:  11335
round:  2502
bounce: 12
state visited:  11335
round:  2503
bounce: 12
state visited:  11337
round:  2504
bounce: 12
state visited:  11348
round:  2505
bounce: 19
state visited:  11349
round:  2506
bounce: 12
state visited:  11351
round:  2507
bounce: 19
state visited:  11352
round:  2508
bounce: 8
state visited:  11353
round:  2509
bounce: 29
state visited:  11356
roun

bounce: 7
state visited:  11752
round:  2676
bounce: 26
state visited:  11757
round:  2677
bounce: 26
state visited:  11758
round:  2678
bounce: 7
state visited:  11759
round:  2679
bounce: 5
state visited:  11759
round:  2680
bounce: 39
state visited:  11762
round:  2681
bounce: 5
state visited:  11762
round:  2682
bounce: 31
state visited:  11762
round:  2683
bounce: 31
state visited:  11764
round:  2684
bounce: 7
state visited:  11764
round:  2685
bounce: 44
state visited:  11766
round:  2686
bounce: 19
state visited:  11766
round:  2687
bounce: 7
state visited:  11766
round:  2688
bounce: 12
state visited:  11766
round:  2689
bounce: 15
state visited:  11766
round:  2690
bounce: 9
state visited:  11766
round:  2691
bounce: 21
state visited:  11767
round:  2692
bounce: 26
state visited:  11773
round:  2693
bounce: 11
state visited:  11773
round:  2694
bounce: 7
state visited:  11773
round:  2695
bounce: 65
state visited:  11776
round:  2696
bounce: 24
state visited:  11776
round:  2

bounce: 21
state visited:  12124
round:  2863
bounce: 38
state visited:  12124
round:  2864
bounce: 9
state visited:  12125
round:  2865
bounce: 7
state visited:  12125
round:  2866
bounce: 24
state visited:  12129
round:  2867
bounce: 12
state visited:  12130
round:  2868
bounce: 10
state visited:  12130
round:  2869
bounce: 26
state visited:  12131
round:  2870
bounce: 28
state visited:  12132
round:  2871
bounce: 40
state visited:  12134
round:  2872


In [ ]:
print("average bouncing per game: ", total_num_bounce / 100)

print("totoal state visited after training: ", np.count_nonzero(Q_table))



In [ ]:
print (sorted(Q_table[np.nonzero(Q_table)],reverse=True), '\n')

In [ ]:
print (sorted(N_sa[np.nonzero(N_sa)],reverse=True), '\n')

In [ ]:
# print (N_sa[np.nonzero(N_sa)])

import statistics as stat

In [ ]:
data = N_sa[np.nonzero(N_sa)]

In [ ]:
print (stat.mean(data), stat.median(data), stat.mode(data))

print (stat.median_low(data), stat.median_high(data))

print (stat.median_grouped(data))

print (stat.harmonic_mean(data))

In [ ]:
print(stat.stdev(data))

In [ ]:
def Q_agent_test(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q


    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f_test(current_state)
    
    r = reward(current_state)
    
    return a

In [ ]:
def argmax_f_test(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up_test(current_state))
    #print ('up' , (b_x ,b_y, v_x, v_y, p))
    f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down_test(current_state))
    #print ('down', (b_x ,b_y, v_x, v_y, p))

    f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move_test(current_state))
    #print ('nothing',(b_x ,b_y, v_x, v_y, p))

    f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [ ]:
class enemy():
    position = 0.4
    height = 0.2
    speed = 0.02
    center = (position + height) / 2
    
    def __init__(self, pos):
        self.position = pos
        
    
    def up(self):
        self.position = max(self.position - self.speed, 0)
        #return self.position

    def down(self):
        self.position = min(self.position + self.speed, 0.8)
        #return self.position
    
    
    def getpos(self):
        return self.position
    
    

In [ ]:
class Test_State(State):
    terminate_left = False
    terminate_right = False


    def update(self):
        global num_bounce
        global catch
        
        self.ball_x += self.velocity_x
        self.ball_y += self.velocity_y
        
        if self.ball_x >= 1:        # hitting right bound
            
            
            if self.ball_y >= self.paddle_y and self.ball_y <= self.paddle_y + 0.2:    # hitting paddle
                self.ball_x = 2 - self.ball_x
                #print("bounce")
                num_bounce += 1
#                 if curr_state not in state_tracker:
#                     #print(curr_state)
#                     state_tracker.append(curr_state)
                                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
                    
                
                
            else:    # missing paddle
                self.ball_x =  2 - self.ball_x
#                 print("ai loss")
                self.terminate_right = True

            
        if self.ball_x <= 0:     # hitting left bound
#             print("neg x pos")
            if self.ball_y >= e.position and self.ball_y <= e.position + 0.2:    # hitting paddle
                self.ball_x = - self.ball_x

                catch += 1
                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
            else:
                self.ball_x =  - self.ball_x
#                 print("enemy loss")
                self.terminate_left = True
                    
        if self.ball_y <= 0:     # hitting upper bound
            self.ball_y = -self.ball_y
            self.velocity_y = -self.velocity_y
        if self.ball_y  >= 1:    # hitting right bound
            self.ball_y = 2 - self.ball_y
            self.velocity_y = -self.velocity_y
        
        y_diff = self.ball_y - e.center

        if y_diff < 0:
            e.up()
        elif y_diff > 0:
            e.down()
        
        #print ("self.ball_x is: ", self.ball_x)
            

In [ ]:

def get_up_test(current_state):
    #print("up current", match(current_state))
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.up()
    #print("new state", match(new_state))
    return new_state

def get_down_test(current_state):
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.down()
    return new_state

def get_no_move_test(current_state):
    new_state = Test_State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.update()
    return new_state

In [ ]:
t_ = 1
num_bounce = 0
total_num_bounce = 0

catch = 0

count_ai_win = 0
count_enemy_win = 0

for i in range(1000):
    s = Test_State(0.5, 0.5, 0.03, 0.01, 0.4)
    e = enemy(0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    catch = 0
    while s.terminate_left is False and s.terminate_right is False:
        
        next_action = Q_agent_test(current_state, t_)

        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    
    if (s.terminate_left): 
        
        print("AI win")
        count_ai_win += 1
    elif(s.terminate_right):
        print("enemy win")
        count_enemy_win += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))


In [ ]:
print("average bouncing per game: ", total_num_bounce / 1000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))

In [ ]:
print("ai win times is: ", count_ai_win)
print("enemy win times is: ", count_enemy_win)


In [ ]:
print("ai winning rate is: ", count_ai_win / 1000)